In [1]:
import astropy.table as apt

In [3]:
t=apt.Table.read('../../galstreams/galstreams/tracks/track.st.GD-1.pricewhelan2018.ecsv')

In [4]:
t2 = apt.Table.read('../../galstreams/galstreams/tracks/track.st.GD-1.pricewhelan2018.summary.ecsv')

In [5]:
t.write('GD-1.pricewhelan2018.asdf',data_key='track',format='asdf')

Exception: The asdf module is required to read and write ASDF files

In [ ]:
apt.Table.write()